## тренировка Титаника через Keras и tf.data.Dataset

    задача - сделать flow так, чтобы pandas даже не вылезал - предобработку фичей целеком через tf.data
    

In [88]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import RMSprop

%matplotlib inline

In [89]:
BATCH_SIZE = 16

# столбцы и таргет можно было бы уже и здесть отобрать, но хочу руками поковырять
dataset = tf.data.experimental.make_csv_dataset('data/titanic-train.csv', BATCH_SIZE)

In [90]:
for batch in dataset.take(1):
    print(batch)

OrderedDict([('PassengerId', <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([311, 552, 387, 245, 814, 392, 618, 524,  16, 487, 424, 449, 831,
       461, 712,   6], dtype=int32)>), ('Survived', <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0], dtype=int32)>), ('Pclass', <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 2, 3, 3, 3, 3, 3, 1, 2, 1, 3, 3, 3, 1, 1, 3], dtype=int32)>), ('Name', <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'Hays, Miss. Margaret Bechstein', b'Sharp, Mr. Percival James R',
       b'Goodwin, Master. Sidney Leonard', b'Attalah, Mr. Sleiman',
       b'Andersson, Miss. Ebba Iris Alfrida', b'Jansson, Mr. Carl Olof',
       b'Lobb, Mrs. William Arthur (Cordelia K Stanlick)',
       b'Hippach, Mrs. Louis Albert (Ida Sophia Fischer)',
       b'Hewlett, Mrs. (Mary D Kingcome) ',
       b'Hoyt, Mrs. Frederick Maxfield (Jane Anne Forby)',
       b'Danbom, Mrs. Ernst Gilbert (Anna Sigrid Maria Brogren)',
  

In [91]:
FEATURE_COLUMNS = ['Age', 'Sex', 'Pclass', 'Fare']
TARGET_COLUMN = 'Survived'

# 1-й фильтр - отбор столбцов
def filter_columns(row_data):
    """
    на входе - 1 батч, на выходе - бач с усеченным набором столбцов и отдельно лейбл
    """
    label = row_data.pop(TARGET_COLUMN)
    features = row_data
    
    # нужно брать именно список - чтобы зафиксировать ключи. иначе будет ругаться
    # что ключи поменялись во время итерирования
    keys = list(features.keys())
    
    
    for column in keys:
        if column not in FEATURE_COLUMNS:
            features.pop(column)
    return features, label

In [92]:
dataset = dataset.map(filter_columns)

for features, label in dataset.take(1):
    print(features)


OrderedDict([('Pclass', <tf.Tensor: shape=(16,), dtype=int32, numpy=array([3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 2, 2, 2, 1, 3, 3], dtype=int32)>), ('Sex', <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'female', b'male', b'male', b'female', b'female', b'male',
       b'male', b'male', b'male', b'female', b'female', b'male', b'male',
       b'female', b'male', b'female'], dtype=object)>), ('Age', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([31.  ,  0.  , 40.  , 28.  ,  0.  , 25.  , 29.  , 22.  ,  0.92,
       16.  , 32.  ,  0.83,  0.  , 30.  , 26.  , 22.  ], dtype=float32)>), ('Fare', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([  7.8542,   7.3125,  27.7208,   7.8958,   7.225 ,  17.8   ,
         7.0458,   8.05  , 151.55  ,   7.7333,  13.    ,  29.    ,
         0.    ,  93.5   ,  14.4542,   7.75  ], dtype=float32)>)])


In [93]:
# Можно его еще закешить для ускорения, зашаффлить и задать сколько раз его крутить на эпоху

In [94]:
def mix_preparation(dataset):
    return dataset.shuffle(100).repeat(3).cache()

dataset = mix_preparation(dataset)

In [99]:
# создаем входной слой. он состоит из функций, которые едят массивы прямо из срезов
# (срез - это значения ключа на батче)

CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : [1, 2, 3],
}

def create_input_layer():
    
    # считаем что среднее и стд упали откуда-то с неба, сейчас это не суть
    age = tf.feature_column.numeric_column('Age', 
                                        normalizer_fn=lambda x: (x-29.699118)/ 14.526497, 
                                        shape=[1],
                                        default_value=0)
    fare = tf.feature_column.numeric_column('Fare', 
                                        normalizer_fn=lambda x: (x-32.204208)/ 49.693429, 
                                        shape=[1],
                                        default_value=0)
    
    sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex', CATEGORIES['sex'],default_value=0)
    sex = tf.feature_column.indicator_column(sex)

    cls = tf.feature_column.categorical_column_with_vocabulary_list('Pclass', CATEGORIES['class'],default_value=0)
    cls = tf.feature_column.indicator_column(cls)
    
    return tf.keras.layers.DenseFeatures([age, fare, sex, cls])

In [100]:
# input_layer - это функция на баче. чтобы проверить, нужно извлечь бач и применить к нему
input_layer = create_input_layer()

batch, label = next(iter(dataset))
input_layer(batch), label

(<tf.Tensor: shape=(16, 7), dtype=float32, numpy=
 array([[-0.4611654 , -0.3864537 ,  0.        ,  1.        ,  0.        ,
          1.        ,  0.        ],
        [ 0.36491126,  1.0317419 ,  1.        ,  0.        ,  0.        ,
          0.        ,  1.        ],
        [-0.32348594,  2.4016414 ,  1.        ,  0.        ,  0.        ,
          0.        ,  1.        ],
        [ 2.2235837 , -0.11378183,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ],
        [ 0.02071266, -0.3864537 ,  0.        ,  1.        ,  0.        ,
          1.        ,  0.        ],
        [ 1.190988  , -0.46694723,  0.        ,  0.        ,  1.        ,
          1.        ,  0.        ],
        [ 0.1583921 , -0.03429444,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ],
        [ 0.70910984,  2.0585377 ,  1.        ,  0.        ,  0.        ,
          0.        ,  1.        ],
        [-0.5988448 , -0.49243754,  0.        ,  0.        ,  1.      

## Еще раз все вместе

     1. можно создать предобработку прям датасета. т.е на входе батч, на выходе батч
     или бач + метки, как угодно. на входе на самом деле одеред дикт 
     
     2. потом можно написать функцию, которая на входе получает ордеред дикт, а потом все 
     это уплотнить в слой. И получится прекрасный входной слой

In [101]:
model = tf.keras.models.Sequential([
    input_layer,
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr = 0.001),
              metrics = ['accuracy'])

history = model.fit(dataset, epochs=5,steps_per_epoch=5000)

Epoch 1/5
5000/5000 [==============================] - 39s 8ms/step - loss: 0.4151 - accuracy: 0.8189
Epoch 2/5
5000/5000 [==============================] - 39s 8ms/step - loss: 0.3780 - accuracy: 0.8379
Epoch 3/5
5000/5000 [==============================] - 39s 8ms/step - loss: 0.3590 - accuracy: 0.8461
Epoch 4/5
5000/5000 [==============================] - 52s 10ms/step - loss: 0.3463 - accuracy: 0.8540
Epoch 5/5
5000/5000 [==============================] - 35s 7ms/step - loss: 0.3367 - accuracy: 0.8598
